In [1]:
from urllib.request import urlopen
from urllib.error import HTTPError
import json
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
import statsmodels.api as sm
import seaborn as sns
import matplotlib as mpl
from matplotlib import cycler
import statsmodels.api as sm
import statsmodels.stats.api as sms
import getFamaFrenchFactors as gff
import yfinance as yf
from datetime import datetime, date
import math 
from statsmodels.tsa.stattools import adfuller
from scipy import stats

from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn import metrics

pd.options.mode.chained_assignment = None 

In [45]:
#Insert personal API
api = '123456789'

In [46]:
#Incert the list of stock tickers
stock_one = 'MYNZ'
ticker_list = ['MC.PA']

In [47]:
def growth(ticker, api_key, as_reported=False, limit=130):
    
    if as_reported:
        URL = (f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?limit={limit}&apikey={api_key}")
             
    else:
        URL = (f"https://financialmodelingprep.com/api/v3/financial-growth/{ticker}?limit={limit}&apikey={api_key}")

    response = urlopen(URL)
    data = json.loads(response.read().decode("utf-8"))

    data_formatted = {}
    
    return data

def ratios(ticker, api_key, as_reported=False, limit=130):
    
    if as_reported:
        URL = (f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?limit={limit}&apikey={api_key}")
             
    else:
        URL = (f"https://financialmodelingprep.com/api/v3/ratios/{ticker}?limit={limit}&apikey={api_key}")

    response = urlopen(URL)
    data = json.loads(response.read().decode("utf-8"))

    data_formatted = {}
    

    return data


def key(ticker, api_key, as_reported=False, limit=130):
    
    if as_reported:
        URL = (f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?limit={limit}&apikey={api_key}")
             
    else:
        URL = (f"https://financialmodelingprep.com/api/v3/key-metrics/{ticker}?limit={limit}&apikey={api_key}")

    response = urlopen(URL)
    data = json.loads(response.read().decode("utf-8"))

    data_formatted = {}
    

    return data


def quote(ticker, api_key, as_reported=False):
    
    if as_reported:
        URL = (f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?serietype=line&apikey={api_key}")
             
    else:
        URL = (f"https://financialmodelingprep.com/api/v3/historical-price-full/{ticker}?serietype=line&apikey={api_key}")

    response = urlopen(URL)
    data = json.loads(response.read().decode("utf-8"))

    data_formatted = {}

    return data

In [48]:
#Download financial data - Growth and Ratios
df_initial1 = growth(stock_one, api, as_reported=False, limit=130)
df_initial1 = pd.json_normalize(df_initial1)

df_initial2 = key(stock_one, api, as_reported=False, limit=130)
df_initial2 = pd.json_normalize(df_initial2)
    
for stock in ticker_list:
    df_new1 = growth(stock, api, as_reported=False, limit=130)
    df_new1 = pd.json_normalize(df_new1)
    df_initial1 = pd.concat([df_new1,df_initial1],axis = 0)
    
for stock in ticker_list:
    df_new2 = key(stock, api, as_reported=False, limit=130)
    df_new2 = pd.json_normalize(df_new2)
    df_initial2 = pd.concat([df_new2,df_initial2],axis = 0)

In [49]:
df_initial1['name'] = df_initial1['symbol'] + df_initial1['date']
df_initial1['date'] = pd.to_datetime(df_initial1['date'])

df_initial2['name'] = df_initial2['symbol'] + df_initial2['date']
df_initial2['date'] = pd.to_datetime(df_initial2['date'])

df_initial3 = pd.merge(df_initial1, df_initial2, left_on='name', right_on='name')

df_initial3 = df_initial3.rename(columns={"date": "date_x"})
df_initial3 = df_initial3.rename(columns={"symbol": "symbol_x"})
df_initial3 = df_initial3.rename(columns={"period": "period_x"})

In [50]:
#Download stock data 
FIELDS = ['close','date']

df_initial2 = quote(stock_one, api, as_reported=False)
df_initial2 = pd.json_normalize(df_initial2['historical'])
df_initial2['symbol_x'] = stock_one

for stock in ticker_list:
    try:
        df_new2 = quote(stock, api, as_reported=False)
        df_new2 = pd.json_normalize(df_new2['historical'])
        df_new2 = df_new2[FIELDS]
        df_new2['symbol_x'] = stock 
        df_initial2 = pd.concat([df_new2,df_initial2],axis = 0)
    except:
        pass

In [51]:
#Merge and clean data
df_initial2 = df_initial2.rename(columns={"date": "date_x"})
df_initial2 = df_initial2.loc[:, ['symbol_x','date_x','close']]

df_1 = df_initial2[(df_initial2['date_x']>='2002-01-01') & (df_initial2['date_x']<='2022-03-31')]
df_2 = df_initial3[(df_initial3['date_x']>='2002-01-01') & (df_initial3['date_x']<='2022-03-31')]

df_1['close'] = df_1.groupby('symbol_x')['close'].shift(-1)

df_1['date_x'] = pd.to_datetime(df_1['date_x'])
df_2['date_x'] = pd.to_datetime(df_2['date_x'])

df_2['year'] = pd.DatetimeIndex(df_2['date_x']).year
df_2['month'] = pd.DatetimeIndex(df_2['date_x']).month

df = pd.merge_asof(df_1.sort_values('date_x'),df_2.sort_values('date_x'), on='date_x', by='symbol_x', tolerance=pd.Timedelta("2 day"))

df1 = df.dropna(subset=['period_x']).sort_values(['symbol_x','date_x'])
df1 = df.dropna(subset=['period_x'])

df = df.drop_duplicates(subset=['currentRatio'], keep='first')

In [52]:
#Download data on the local. Further, the data is uploaded in repository
df.to_csv('/Users/daniil_bruk/Desktop/Main_EU.csv', index = True, header=True)
df_2.to_csv('/Users/daniil_bruk/Desktop/Main_financials_EU.csv', index = True, header=True)
df_1.to_csv('/Users/daniil_bruk/Desktop/Main_prices_EU.csv', index = True, header=True)

## Data Preparation

In [2]:
#Upload data from the repository
df = pd.read_csv('Main_EU.csv')
df_1 = pd.read_csv('Main_prices_EU.csv')
df_2 = pd.read_csv('Main_financials_EU.csv')

#Clear and format data
df_m = df_1.sort_values(by= ['symbol_x','date_x'],ascending = True)

df_m['date_x'] = pd.to_datetime(df_m['date_x'])
df_m = df_m.set_index(['date_x'])

df_m = df_m.groupby('symbol_x')['close'].resample('M').last().pct_change().dropna()

df_m = pd.DataFrame(df_m)
df_m = df_m.reset_index()

df_m['day'] = pd.DatetimeIndex(df_m['date_x']).day
df_m['month'] = pd.DatetimeIndex(df_m['date_x']).month

df2 = df_1

df2['day'] = pd.DatetimeIndex(df2['date_x']).day
df2['month'] = pd.DatetimeIndex(df2['date_x']).month
df2['year'] = pd.DatetimeIndex(df2['date_x']).year 

#Extract last price of the year
df_dec = df2.loc[(df2['month'] == 12)]
df_dec = df_dec.drop_duplicates(subset=['symbol_x','year'], keep='first')

df_dec = df_dec[['symbol_x','close','year']]

df = df.dropna(subset=['year'])

df['year'] = df['year'].astype('int64')

#Divide into two sub-samples based on the month for the consistency
df_m_1 = df_m[df_m['month'] > 3]
df_m_1['year'] = pd.DatetimeIndex(df_m_1['date_x']).year - 1
df_m_2 = df_m[df_m['month'] <= 3]
df_m_2['year'] = pd.DatetimeIndex(df_m_2['date_x']).year - 2

#Create final sample
df_12_1 = pd.merge_asof(df_m_1.sort_values('year'),df.sort_values('year'), on='year', by='symbol_x')
df_12_2 = pd.merge_asof(df_m_2.sort_values('year'),df.sort_values('year'), on='year', by='symbol_x')

df_12 = pd.concat([df_12_1, df_12_2], ignore_index=True)

df_12 = pd.merge_asof(df_12.sort_values('year'),df_dec.sort_values('year'), on='year', by='symbol_x')

df_12 = df_12.rename(columns={"close_x": "ret_m"})
df_12 = df_12.rename(columns={"close": "close_dec"})

## Final Data

In [3]:
#Select needed columns
m = df_12[['symbol_x',
'date_x_x',
'year',
'netIncomePerShare',
'bookValuePerShare',
'marketCap',
'ret_m',
'close_dec',
'threeYNetIncomeGrowthPerShare']]

m['month_ind'] = pd.DatetimeIndex(m['date_x_x']).month
m['year'] = pd.DatetimeIndex(m['date_x_x']).year

m = m.rename(columns={"date_x_x": "date", "symbol_x": "ticker", "year": "year_ind"})
m = m.sort_values(by = ['ticker','date'])
m = m.dropna()

m = m[(m['date']>='2007-01-01') & (m['date']<='2022-03-31')]
m = m[(m['netIncomePerShare']> 0) & (m['bookValuePerShare']> 0)]
m = m[(m['threeYNetIncomeGrowthPerShare'] > 0)]

m['pe_dec'] = m['close_dec'] / m['netIncomePerShare']
m['peg'] = m['pe_dec'] / (m['threeYNetIncomeGrowthPerShare'] * 100)

ff3_monthly = pd.read_csv('Europe_3_Factors.csv')

ff3_monthly = ff3_monthly.sort_values(by = ['Date'], ascending=False)

ff3_monthly['Mkt-RF'] = ff3_monthly['Mkt-RF'] / 100
ff3_monthly['SMB'] = ff3_monthly['SMB'] / 100
ff3_monthly['HML'] = ff3_monthly['HML'] / 100
ff3_monthly['RF'] = ff3_monthly['RF'] / 100

m['year'] = pd.DatetimeIndex(m['date']).year
m['month'] = pd.DatetimeIndex(m['date']).month

m = m.merge(ff3_monthly, on=['year','month'])

m = m[['ticker',
'date',
'year_ind',
'month_ind',
'netIncomePerShare',
'threeYNetIncomeGrowthPerShare',
'marketCap',
'ret_m',
'pe_dec',
'peg',
'Mkt-RF',
'SMB',
'HML',
'RF']]

In [31]:
m.corr()

,year_ind,month_ind,netIncomePerShare,threeYNetIncomeGrowthPerShare,marketCap,ret_m,pe_dec,peg,Mkt-RF,SMB,HML,RF
year_ind,1.000000,-0.048778,-0.025150,-0.017490,0.023177,0.051151,0.187107,0.069111,0.067971,0.084140,-0.023723,-0.141364
month_ind,-0.048778,1.000000,0.000747,-0.003011,-0.001092,-0.024244,0.000066,0.002038,0.010594,-0.128212,0.050282,-0.031700
netIncomePerShare,-0.025150,0.000747,1.000000,0.056663,-0.006144,-0.020215,-0.036250,-0.018599,-0.026119,-0.008016,-0.008828,0.011174
threeYNetIncomeGrowthPerShare,-0.017490,-0.003011,0.056663,1.000000,-0.076094,-0.012167,-0.001744,-0.282558,-0.029167,-0.021269,-0.005365,0.091585
marketCap,0.023177,-0.001092,-0.006144,-0.076094,1.000000,-0.012007,0.077148,0.031481,-0.005783,0.000773,-0.000431,0.012966
ret_m,0.051151,-0.024244,-0.020215,-0.012167,-0.012007,1.000000,-0.006940,0.000606,0.437989,0.151482,0.187617,-0.106835
pe_dec,0.187107,0.000066,-0.036250,-0.001744,0.077148,-0.006940,1.000000,0.219070,-0.007683,-0.003011,-0.001935,0.059238
peg,0.069111,0.002038,-0.018599,-0.282558,0.031481,0.000606,0.219070,1.000000,0.011017,0.001946,0.004409,-0.068309
Mkt-RF,0.067971,0.010594,-0.026119,-0.029167,-0.005783,0.437989,-0.007683,0.011017,1.000000,0.066203,0.439217,-0.106573
SMB,0.084140,-0.128212,-0.008016,-0.021269,0.000773,0.151482,-0.003011,0.001946,0.066203,1.000000,-0.101642,-0.194011


In [27]:
ff3_monthly.corr().round(3)

,Mkt-RF,SMB,HML,RF,year,month
Mkt-RF,1.000,-0.106,0.200,-0.042,0.013,0.039
SMB,-0.106,1.000,-0.090,-0.146,0.094,-0.224
HML,0.200,-0.090,1.000,0.064,-0.088,-0.052
RF,-0.042,-0.146,0.064,1.000,-0.760,0.032
year,0.013,0.094,-0.088,-0.760,1.000,-0.045
month,0.039,-0.224,-0.052,0.032,-0.045,1.000


## Descriptive statistics

In [4]:
ff = m

ff['peg'] = ff['peg'].astype('float64')
ff['pe_dec'] = ff['pe_dec'].astype('float64')
ff['netIncomePerShare'] = ff['netIncomePerShare'].astype('float64')
ff['threeYNetIncomeGrowthPerShare'] = ff['threeYNetIncomeGrowthPerShare'].astype('float64')
ff = ff[['netIncomePerShare','threeYNetIncomeGrowthPerShare','marketCap','ret_m','pe_dec','peg']]

gg = ff.describe()
gg.round(5)

,netIncomePerShare,threeYNetIncomeGrowthPerShare,marketCap,ret_m,pe_dec,peg
count,19009.00000,19009.00000,1.900900e+04,19009.00000,19009.00000,19009.00000
mean,5.50167,0.73839,1.054352e+10,0.00714,19.25657,1.10298
std,46.61876,0.78122,2.296585e+10,0.09111,16.50644,2.81735
min,0.01744,0.00402,4.152600e+06,-0.67734,0.92329,0.00440
25%,1.19628,0.22183,2.670593e+08,-0.03831,11.17703,0.15605
50%,2.30596,0.46797,1.156794e+09,0.00554,16.08713,0.35429
75%,4.24308,0.93244,8.983087e+09,0.05234,22.59405,0.84813
max,1816.77573,3.98432,2.007340e+11,1.64444,326.89595,33.21967


In [5]:
ff3_monthly_2 = ff3_monthly[(ff3_monthly['year'] > 2007)]*100
ff3_monthly_2.describe().iloc[:,[0,1,2,3]].round(5)

,Mkt-RF,SMB,HML,RF
count,172.0000,172.00000,172.00000,172.00000
mean,0.3443,0.14814,-0.18587,0.04215
std,5.6882,1.85823,2.86478,0.06480
min,-22.0200,-4.91000,-11.30000,0.00000
25%,-2.7575,-1.13500,-1.90000,0.00000
50%,0.5850,0.17000,-0.43000,0.01000
75%,4.3050,1.46750,1.23000,0.06250
max,16.6200,5.04000,12.09000,0.21000


## Raw return analysis

In [6]:
years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)

bin_labels_5 = ['P1', 'P2', 'P3']

s = m[m['year_ind']==2007]
s['quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
monthly = s.groupby(['month_ind','quantile']).mean()

years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)[1:]

for year in years:
    s = m[m['year_ind']== year]
    s['quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
    g = s.groupby(['month_ind','quantile']).mean()
    monthly = pd.concat([g,monthly])

df_mean = pd.DataFrame(monthly)
df_mean['ret_m'] = df_mean['ret_m'] * 100
df_mean = df_mean.reset_index()

In [9]:
df_mean.groupby(['quantile']).mean().round(5).iloc[:,[2,3,4,5,6,7]]

,netIncomePerShare,threeYNetIncomeGrowthPerShare,marketCap,ret_m,pe_dec,peg
quantile,,,,,,
P1,11.52510,1.83463,6.131130e+09,0.96413,10.78643,0.06950
P2,4.00069,0.56343,1.047041e+10,0.72470,19.19253,0.44481
P3,3.08924,0.12116,1.517588e+10,0.67170,26.13117,4.29152


In [10]:
df_mean.groupby('quantile').std().round(5).iloc[:,[2,3,4,5,6,7]]

,netIncomePerShare,threeYNetIncomeGrowthPerShare,marketCap,ret_m,pe_dec,peg
quantile,,,,,,
P1,22.81516,0.19561,2.843213e+09,5.74636,2.59493,0.01307
P2,1.21656,0.12472,2.748253e+09,4.60692,4.21611,0.10832
P3,0.83571,0.05330,6.588857e+09,4.26513,8.70424,1.87913


In [11]:
#data for the quantile total return graphs
df_graph = df_mean.groupby(['year_ind','month_ind','quantile']).mean().round(5).iloc[:,[3]]
df_graph.to_csv('/Users/daniil_bruk/Desktop/graph_EU.csv', index = True, header=True)

In [12]:
#data for quantile returns histogram by years
hist_returns = df_mean.groupby(['year_ind','quantile'])['ret_m'].mean()
hist_returns = hist_returns.reset_index()
hist_returns.to_csv('/Users/daniil_bruk/Desktop/hist_wp_eu.csv', index = True, header=True)

hist_returns_3= df_mean.groupby(['year_ind'])['Mkt-RF'].mean()
hist_returns_3 = hist_returns_3.reset_index()

In [13]:
#Sharpe ratio
sharpe = 12 * (df_mean.groupby(['quantile'])['ret_m'].mean() - df_mean.groupby(['quantile'])['RF'].mean()) / (df_mean.groupby(['quantile'])['ret_m'].std() * math.sqrt(12))
pd.DataFrame(sharpe)

,0
quantile,
P1,0.580822
P2,0.544445
P3,0.545024


In [14]:
#Market sharpe ratio
sharpe_market = 12 * (df_mean['Mkt-RF'].mean()) / (df_mean['Mkt-RF'].std() * math.sqrt(12))
sharpe_market

0.2547248247094949

In [15]:
#M2
m_squared = sharpe * (df_mean['Mkt-RF'].std() * math.sqrt(12)) + df_mean['RF'].mean()
pd.DataFrame(m_squared.round(5))

,0
quantile,
P1,0.11213
P2,0.10515
P3,0.10526


In [17]:
#PE / PEG matrix
years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)

bin_labels_5 = ['P1', 'P2', 'P3']

s = m[m['year_ind']==2007]
s['PEG_quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
s['PE_quantile'] = pd.qcut(s['pe_dec'], q=[0, .2, .8, 1],labels=bin_labels_5)

monthly = s.groupby(['PEG_quantile','PE_quantile']).mean()

years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)[1:]

for year in years:
    s = m[m['year_ind']== year]
    s['PEG_quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
    s['PE_quantile'] = pd.qcut(s['pe_dec'], q=[0, .2, .8, 1],labels=bin_labels_5)
    g = s.groupby(['PEG_quantile','PE_quantile']).mean()
    monthly = pd.concat([g,monthly])

df_mean_2 = pd.DataFrame(monthly['ret_m'])

df_mean_2 = df_mean_2.groupby(['PEG_quantile','PE_quantile']).mean().round(5).unstack()
df_mean_2.round(5)

ret_m                  
PE_quantile        P1       P2       P3
PEG_quantile                           
P1            0.00726  0.01295  0.00630
P2            0.00683  0.00501  0.00208
P3            0.01913  0.00466  0.00551

In [19]:
#t-test on mean equivalence
df_lowest = df_mean[(df_mean['quantile'] == 'P1')]
df_highest = df_mean[(df_mean['quantile'] == 'P3')]
df_middle = df_mean[(df_mean['quantile'] == 'P2')]


stats.ttest_ind(df_final['factor_U'], df_final['factor_O'], equal_var=False)

Ttest_indResult(statistic=0.5505791633099856, pvalue=0.5822881878958022)

## Creating PEG - factor values

In [18]:
years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)

bin_labels_5 = ['P1', 'P2', 'P3']

s = m[m['year_ind']==2007]
s['quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
monthly = s.groupby(['month_ind','quantile']).mean()

years = m['year_ind'].drop_duplicates()
years = years.sort_values(ascending = True)[1:]

for year in years:
    s = m[m['year_ind']== year]
    s['quantile'] = pd.qcut(s['peg'], q=[0, .2, .8, 1],labels=bin_labels_5)
    g = s.groupby(['month_ind','quantile']).mean()
    monthly = pd.concat([g,monthly])
    

df_mean = pd.DataFrame(monthly)
 
df_mean = df_mean.reset_index()

df_lowest = df_mean[(df_mean['quantile'] == 'P1')]
df_highest = df_mean[(df_mean['quantile'] == 'P3')]
df_middle = df_mean[(df_mean['quantile'] == 'P2')]

df_final = df_lowest
df_final['factor_UMO'] = df_lowest['ret_m'].values - df_highest['ret_m'].values - df_final['RF']
df_final['factor_U'] = df_lowest['ret_m'].values - df_final['RF']
df_final['factor_O'] = df_highest['ret_m'].values - df_final['RF']
df_final['factor_M'] = df_middle['ret_m'].values - df_final['RF']
df = df_final

## Tests 

In [20]:
#find p-values for all
y = df['factor_UMO']
x = df[['Mkt-RF','SMB','HML']]

x = sm.add_constant(x)
results = sm.OLS(y,x).fit()

test_result = sms.het_breuschpagan(results.resid, results.model.exog)
test_result[1]

0.130055104355756

In [21]:
y1 = df['factor_U']
y2 = df['factor_M']
y3 = df['factor_O']
y4 = df['factor_UMO']

y = {'1':y1,'2':y2,'3':y3,'4':y4}

y = pd.DataFrame(y)
result = adfuller(y['4'])
result[1]

8.1509093044039685e-25

## Regression Analysis

In [23]:
#OLS Regression - Three-Factor
y = df['factor_O']
x = df[['Mkt-RF','SMB','HML']]

x = sm.add_constant(x)
results = sm.OLS(y,x).fit(cov_type='HC1') #HC1 if needed
results.summary() 

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:               factor_O   R-squared:                       0.698
Model:                            OLS   Adj. R-squared:                  0.693
Method:                 Least Squares   F-statistic:                     81.95
Date:                Sun, 05 Jun 2022   Prob (F-statistic):           2.07e-33
Time:                        18:35:36   Log-Likelihood:                 427.01
No. Observations:                 183   AIC:                            -846.0
Df Residuals:                     179   BIC:                            -833.2
Df Model:                           3                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0027      0.002      1.483      0.138      -0.001       0.006
Mkt-RF         0.6447      0.043     14.877      0.000       0.560       0.730
SMB            0.4682      0.103      4.531      0.000       0.266       0.671
HML           -0.1520      0.081     -1.873      0.061      -0.311       0.007
==============================================================================
Omnibus:                        2.312   Durbin-Watson:                   2.101
Prob(Omnibus):                  0.315   Jarque-Bera (JB):                2.007
Skew:                          -0.133   Prob(JB):                        0.367
Kurtosis:                       3.438   Cond. No.                         54.7
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""

In [24]:
#R2 for all regressions in turn
results.rsquared

0.6982337037978674

In [58]:
#OLS Regression - CAPM
y = df['factor_UMO']
x = df[['Mkt-RF']]

x = sm.add_constant(x)
results = sm.OLS(y,x).fit(cov_type='HC1') #HC1 if needed
results.summary()  

<class 'statsmodels.iolib.summary.Summary'>
"""
                            OLS Regression Results                            
==============================================================================
Dep. Variable:             factor_UMO   R-squared:                       0.201
Model:                            OLS   Adj. R-squared:                  0.188
Method:                 Least Squares   F-statistic:                     17.08
Date:                Sun, 29 May 2022   Prob (F-statistic):           8.42e-10
Time:                        13:10:13   Log-Likelihood:                 372.73
No. Observations:                 183   AIC:                            -737.5
Df Residuals:                     179   BIC:                            -724.6
Df Model:                           3                                         
Covariance Type:                  HC1                                         
==============================================================================
                 coef    std err          z      P>|z|      [0.025      0.975]
------------------------------------------------------------------------------
const          0.0025      0.002      1.019      0.308      -0.002       0.007
Mkt-RF         0.0601      0.068      0.879      0.379      -0.074       0.194
SMB            0.4253      0.166      2.566      0.010       0.100       0.750
HML            0.4582      0.116      3.966      0.000       0.232       0.685
==============================================================================
Omnibus:                        7.294   Durbin-Watson:                   2.219
Prob(Omnibus):                  0.026   Jarque-Bera (JB):               12.897
Skew:                           0.030   Prob(JB):                      0.00158
Kurtosis:                       4.299   Cond. No.                         54.7
==============================================================================

Notes:
[1] Standard Errors are heteroscedasticity robust (HC1)
"""